# Imports

In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Reading Data

In [ ]:
import json
f = open('/content/drive/MyDrive/hypd/content_masked.json')
 
# returns JSON object as
# a dictionary
content = json.load(f)



In [ ]:
len(content)

7581

In [ ]:
content[7]

{'_id': {'$oid': '*****************bbdc'},
 'brand_ids': [{'$oid': '*****************1058'}],
 'caption': 'Keeps you active and energetic throughout the day\n#fast&up #workout #supplement #energy',
 'catalog_ids': [{'$oid': '5f92b20f40006497ff42106b'},
  {'$oid': '5f9d63bb3ac6177434f48029'},
  {'$oid': '5f93e4a040006497ff4210dd'}],
 'catalog_info': None,
 'category_path': ['/615d825d528b706ef3882b2b/615ea667528b706ef3883322'],
 'comment_count': 0,
 'created_at': {'$date': '2021-04-21T21:35:05.719Z'},
 'hashtags': ['#fast', '#workout', '#supplement', '#energy'],
 'influencer_ids': [{'$oid': '*****************033e'}],
 'is_active': True,
 'is_processed': True,
 'label': {'age_groups': ['26-30', '30-35'],
  'genders': ['M', 'F'],
  'interests': ['workout',
   'gym',
   'vitamin',
   'energy',
   'protien',
   'hydration']},
 'last_sync': {'$date': '2022-02-12T09:34:43.513Z'},
 'like_count': 4,
 'like_ids': [{'$oid': '*****************bee1'},
  {'$oid': '*****************0f4b'},
  {'$oid':

In [ ]:
import json
f = open('/content/drive/MyDrive/hypd/catalog_masked.json')
 
# returns JSON object as
# a dictionary
catalog = json.load(f)



In [ ]:
catalog[6]

{'_id': {'$oid': '*****************de5d'},
 'base_price': 1949,
 'brand_id': {'$oid': '*****************ddc1'},
 'cat_one': '*****************7324',
 'cat_three': '*****************f077',
 'cat_two': '*****************7cf9',
 'keywords': ['dress',
  'dresses',
  'maxi dress',
  'maxi',
  'black dress',
  'Embroidered dress',
  'summer dress',
  'black',
  'women',
  'girls',
  'girl',
  'western',
  'western wear',
  'casual',
  'Casuals ',
  'clothing',
  'apparel',
  'apparels',
  'trendy',
  'stylish',
  'brand',
  'vritta'],
 'name': 'Floral Embroidered Maxi Dress - Black',
 'retail_price': 1750,
 'status': 'Publish'}

# Product Recommendation To Influencers

## Data Collection

* Extracting these values from content.json and creating a dataframe.

    * catalog_ids
    * hashtags
    * interests
    * genders
    * caption

* Extracting these values from catalog.json and creating a dataframe.

    * catalog_id
    * name
    * keywords

In [ ]:
content_dict = {
    'catalog_id':[],
    'hashtags':[],
    'interests':[],
    'genders':[],
    'caption':[]
}
for i in range(1,len(content)-1):
    # print(i)
    # print("i-",i)
    if content[i]['catalog_ids']!=None:
        content_dict['catalog_id'].append([j['$oid'] for j in content[i]['catalog_ids']])
        if content[i]['hashtags']!=None:
            content_dict['hashtags'].append([k.replace('#','') for k in content[i]['hashtags']])
        else:
            content_dict['hashtags'].append([None])
   
        if content[i]['label']!=None:
            if content[i]['label']['interests']!=None:
                content_dict['interests'].append(content[i]['label']['interests'])
            else:
                content_dict['interests'].append(None)
            genders = content[i]['label']['genders']
            if genders!=None:
                for j in range(len(genders)):
                    if genders[j].lower()=='f':
                        genders[j]='girl'
                    elif genders[j].lower()=='m':
                        genders[j]='boy'
                content_dict['genders'].append(genders)
            else:
                content_dict['genders'].append([None])
        else:
            print('No value found')
            content_dict['genders'].append([None])
            content_dict['interests'].append([None])
        content_dict['caption'].append( content[i]['caption'] )

No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value found
No value f

In [ ]:
print(len(content_dict['hashtags']))
print(len(content_dict['catalog_id']))
print(len(content_dict['interests']))
print(len(content_dict['genders']))
print(len(content_dict['caption']))
print(len(content_dict))

5843
5843
5843
5843
5843
5


* arrays lens should be equal to create df of dict of arrays.

In [ ]:
content_df = pd.DataFrame(content_dict)
content_df.head(10)

,catalog_id,hashtags,interests,genders,caption
0,[6035106e58b8f4136b6e4505],"[beardoil, package, gift, sample, pebbles]","[Sample, testers, free products]","[boy, girl]",A curated box of surprise samples from select ...
1,"[60094d9352e78c76b023fc9b, 6008556952e78c76b02...","[quaclothing, workwear, corporatefashion, offi...","[work wear, work, women, office look, professi...",[girl],Ways to Wear Black Cotton Trousers\n\nThe tape...
2,"[5fbbbb65c94eac4d3ca460cd, 5fbcc096c94eac4d3ca...","[kicaactive, workout, gymwear, activewear]","[work out, activewear, gym, workout, body toni...",[girl],She's anything but ordinary. Wearing Kica Acti...
3,[5fda294b0a30d919d2439684],"[destello, fashion, trending, ootd, shirt]","[shirt, graphic shirt, quirky, orinted shirt, ...",[boy],Midweek feels. There's so much more to a week ...
4,"[602a1a177f064048a3e7457d, 602a0df67f064048a3e...","[mensclothing, floralprint, printedshirt, beac...","[tailored , custom, printed shirts, casual, ev...",[boy],"Vibrant Tropical Shirt from 15 buttons, styled..."
5,"[5f92e8fc40006497ff4210a3, 5f93cebf40006497ff4...","[fast, supplements, energy, gym]","[work out, gym, supplements, protien, energy, ...",[boy],Experience absolute beast mode during workouts...
6,"[5f92b20f40006497ff42106b, 5f9d63bb3ac6177434f...","[fast, workout, supplement, energy]","[workout, gym, vitamin, energy, protien, hydra...","[boy, girl]",Keeps you active and energetic throughout the ...
7,[5fbb77d8c94eac4d3ca46091],"[prolixrbeauty, prolixr, skincare, hydration]","[skincare, beauty, makeup, looking good, routi...",[girl],Keep your skin hydrated with @prolixr All-New ...
8,[5ff585d21beb3405bd1a3ec1],"[kicaactive, leggings, fitness, workoutwear]","[kica active, kica, fitness, gym wear, active ...",[girl],"A hypnotising flow, and nope we aren’t going t..."
9,[5ffebac16a21ca0e6cdd316d],"[halffullhalfempty, spaghettidress, ootd, fash...","[striped dress, casual dress, dress, western w...",[girl],Dresses with pockets >> ! 😍\n#halffullhalfempt...


In [ ]:
catalog_dict = {
    'catalog_id':[],
    'name':[],
    'keywords':[]
}
for i in range(len(catalog)):
    catalog_dict['catalog_id'].append(catalog[i]['_id']['$oid'][-4:])
    catalog_dict['name'].append(catalog[i]['name'])
    catalog_dict['keywords'].append(catalog[i]['keywords'])


In [ ]:
catalog_df = pd.DataFrame(catalog_dict)
catalog_df.head(10)

,catalog_id,name,keywords
0,93d0,Moisturising Cream | Shea Butter & Vitamin E,"[cream, moisturising, shea, butter, vitamin e,..."
1,94cb,Bold Swimsuit,"[swimsuit, gymwear, active wear, swimming cost..."
2,93ff,Onion Seed Hair Oil,"[oil, hair fall, growth oil, hair, onion, blac..."
3,9548,Maroon Embroidered Shift Dress,"[one piece, casual, party, long dress, dress]"
4,9548,Maroon Embroidered Shift Dress,"[one piece, casual, party, long dress, dress]"
5,dcfd,Grey Crop Top With Embroidered Sleeves,"[top, casual wear, trendy, women wear]"
6,de5d,Floral Embroidered Maxi Dress - Black,"[dress, dresses, maxi dress, maxi, black dress..."
7,df1a,Engy Blue Stripe Set,"[coat, coat pant set, office wear, formal wear..."
8,f00d,Taupe Printed Dress With Tie-Up Belt,"[one peice, casual wear, fit and flare dress,..."
9,dd35,Winona Stripe Jumpsuit,"[women apparel , ladies jumpsuit, strip jumpsu..."


## Preprocessing

* Lower texts
* preprocessing text (only for captions)

    * tokenizing
    * removing special characters
    * removing stop words (added few extra words to nltk stop words like today,tommorow etc.,)

* combining these text from content and creating a set (for input)
    * hashtags
    * interests
    * genders
    * caption
    * catalog_tags

In [ ]:
content_df['hashtags'] = content_df['hashtags'].apply(lambda x: [i.lower() for i in x if i!=None])
content_df['interests'] = content_df['interests'].apply(lambda x: [i.lower() for i in x if i!=None])
content_df['caption'] = content_df['caption'].apply(lambda x: x.lower() if (x!=None) else '')


In [ ]:
content_df.iloc[1]['caption']

'ways to wear black cotton trousers\n\nthe tapered pair is cut to a clean, streamlined silhouette with slanted pockets and natural wood buttons. they’re cut from durable cotton woven with stretch so the slim-leg silhouette is flexible. we show you how to style these black pants in 4 different ways!\n#quaclothing #workwear #corporatefashion #officelook'

In [ ]:

stop_words = stopwords.words('english')
stop_words.extend(['get','today','tommorow','yesterday'])

def preprocess_sent(x):
    x = x.lower()
    words = word_tokenize(x)
    # words = [char for char in words if char not in string.punctuation]
    words = [char for char in words if char.isalpha()]

    words = [word for word in words if word not in stop_words]
    return words


In [ ]:
content_df['caption'] = content_df['caption'].apply(lambda x: preprocess_sent(x))

In [ ]:
def unite(x):
    tags = x['hashtags']
    # print(tags)
    inter = x['interests']
    # print(inter)
    gen = x['genders']
    cap = x['caption']

    h = []
    h.extend(tags)
    h.extend(inter)
    h.extend(gen)
    h.extend(cap)
    return list(set(h))


In [ ]:
content_df['new_tags'] = content_df.apply(unite,axis=1)

In [ ]:
train_df = content_df.drop(['hashtags','interests','genders','caption'],axis=1)
train_df.head()

,catalog_id,new_tags
0,[6035106e58b8f4136b6e4505],"[surprise, beardoil, curated, box, boy, sample..."
1,"[60094d9352e78c76b023fc9b, 6008556952e78c76b02...","[wear, slanted, pockets, durable, woven, flexi..."
2,"[5fbbbb65c94eac4d3ca460cd, 5fbcc096c94eac4d3ca...","[pure, activewear, earthy, looking, ordinary, ..."
3,[5fda294b0a30d919d2439684],"[boy, work, orinted shirt, casual, mid week, f..."
4,"[602a1a177f064048a3e7457d, 602a0df67f064048a3e...","[casual, outdoor, printedshirt, men, carefully..."


* adding the catalog name to the contents (using catalog dataset, and last 4 unmasked characters of catalog_ids).

In [ ]:
train_df = train_df.explode('catalog_id')
train_df.head()

,catalog_id,new_tags
0,6035106e58b8f4136b6e4505,"[surprise, beardoil, curated, box, boy, sample..."
1,60094d9352e78c76b023fc9b,"[wear, slanted, pockets, durable, woven, flexi..."
1,6008556952e78c76b023fbed,"[wear, slanted, pockets, durable, woven, flexi..."
1,6009486752e78c76b023fc8b,"[wear, slanted, pockets, durable, woven, flexi..."
2,5fbbbb65c94eac4d3ca460cd,"[pure, activewear, earthy, looking, ordinary, ..."


In [ ]:
train_df.isnull().sum()
train_df.dropna(axis=0,inplace=True)

In [ ]:
def cat_name(cat_ids):
    bad_ids = []
    cat_names = []
    for i in range(len(catalog)):
        names = []
        cat = catalog[i]['_id']
        # print('cat',cat)
        if cat!=None:
            if cat['$oid'][-4:]==cat_ids[-4:]:
                try:
                    cat_names.append(catalog[i]['name'])
                except Exception as e:
                    print(e)
                    cat_names.append(None)
        else:
            print('cat is none')
    return list(set(cat_names))

def flatten_list(lists):
    flat_list = []
    # Iterate through the outer list
    for element in lists:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return list(set(flat_list))

def cat_tags(cat_ids):
    bad_ids = []
    cat_names = []
    for i in range(len(catalog)):
        names = []
        cat = catalog[i]['_id']
        # print('cat',cat)
        if cat!=None:
            if cat['$oid'][-4:]==cat_ids[-4:]:
                cat_names.append(catalog[i]['keywords'])
        else:
            print('cat is none')
    return flatten_list(cat_names)


In [ ]:
train_df['catalog_name'] = train_df['catalog_id'].apply(lambda x: cat_name(x))
train_df['catalog_tags'] = train_df['catalog_id'].apply(lambda x: cat_tags(x))

In [ ]:
train_df.head()

,catalog_id,new_tags,catalog_name,catalog_tags
0,6035106e58b8f4136b6e4505,"[surprise, beardoil, curated, box, boy, sample...",[Look Under The Pebbles 🥰],"[surprise, easter, pebble, samplers, box, pebb..."
1,60094d9352e78c76b023fc9b,"[wear, slanted, pockets, durable, woven, flexi...",[Smart Cotton Crop Trousers - Black],"[office, bottom wear, stylish, fashionista , b..."
1,6008556952e78c76b023fbed,"[wear, slanted, pockets, durable, woven, flexi...",[Ruched Poplin Blouson - Black],"[apparel, stylish, top, fashionista , fashion,..."
1,6009486752e78c76b023fc8b,"[wear, slanted, pockets, durable, woven, flexi...",[Stretch Wool Tailored Suit - Grey ],"[designer, fashion , stylish, boss lady look, ..."
2,5fbbbb65c94eac4d3ca460cd,"[pure, activewear, earthy, looking, ordinary, ...",[Cheer Leggings],"[activewear, gym, leggings, fitness, gymwear]"


In [ ]:
train_df['catalog_name'].iloc[2]

['Ruched Poplin Blouson - Black']

In [ ]:
train_df.isnull().sum()

catalog_id      0
new_tags        0
catalog_name    0
catalog_tags    0
dtype: int64

* Got the names for every catalog_id, great.

In [ ]:
def unite_tags(x):
    new_tags = x['new_tags']
    # print(tags)
    catalog_name = x['catalog_name']
    # print(inter)
    catalog_tags = x['catalog_tags']

    h = []
    h.extend(new_tags)
    h.extend(catalog_name)
    h.extend(catalog_tags)
    return list(set(h))

In [ ]:
train_df['catalog_new_tags'] = train_df.apply(unite_tags,axis=1)
train_df.drop(['new_tags','catalog_tags'],axis=1,inplace=True)
train_df.head()

,catalog_id,catalog_name,catalog_new_tags
0,6035106e58b8f4136b6e4505,[Look Under The Pebbles 🥰],"[surprise, beardoil, curated, box, boy, sample..."
1,60094d9352e78c76b023fc9b,[Smart Cotton Crop Trousers - Black],"[office, bottom wear, wear, slanted, pockets, ..."
1,6008556952e78c76b023fbed,[Ruched Poplin Blouson - Black],"[wear, slanted, pockets, durable, woven, flexi..."
1,6009486752e78c76b023fc8b,[Stretch Wool Tailored Suit - Grey ],"[wear, designer, slanted, pockets, durable, wo..."
2,5fbbbb65c94eac4d3ca460cd,[Cheer Leggings],"[pure, activewear, leggings, earthy, looking, ..."


## Modelling

* creating corpus

In [ ]:
train_df['catalog_new_tags'] = train_df['catalog_new_tags'].apply(lambda x : ' '.join([i for i in x if (i!=None)]))
train_df['catalog_new_tags']

0       surprise beardoil curated box boy samples east...
1       office bottom wear wear slanted pockets durabl...
1       wear slanted pockets durable woven flexible cu...
1       wear designer slanted pockets durable woven fl...
2       pure activewear leggings earthy looking ordina...
                              ...                        
5841    office  face mask outdoor lasting stylish bene...
5841    office outdoor lasting stylish haircare benefi...
5842    office  colored balm Cupid wear outdoor lastin...
5842    myglamm office function Pencil wear outdoor la...
5842    glow myglamm office function lipsticks family ...
Name: catalog_new_tags, Length: 13132, dtype: object

In [ ]:
train_df['catalog_name'] = train_df['catalog_name'].apply(lambda x: ' '.join(i for i in x))
train_df.head()

,catalog_id,catalog_name,catalog_new_tags
0,6035106e58b8f4136b6e4505,Look Under The Pebbles 🥰,surprise beardoil curated box boy samples east...
1,60094d9352e78c76b023fc9b,Smart Cotton Crop Trousers - Black,office bottom wear wear slanted pockets durabl...
1,6008556952e78c76b023fbed,Ruched Poplin Blouson - Black,wear slanted pockets durable woven flexible cu...
1,6009486752e78c76b023fc8b,Stretch Wool Tailored Suit - Grey,wear designer slanted pockets durable woven fl...
2,5fbbbb65c94eac4d3ca460cd,Cheer Leggings,pure activewear leggings earthy looking ordina...


In [ ]:
train_df.iloc[3]['catalog_new_tags']

'wear designer slanted pockets durable woven flexible cut fashion  stylish professional boss lady look show corporatefashion work wear officelook Stretch Wool Tailored Suit - Grey  boss lady silhouette fashionista western wear girls workwear pants formal ways style fashionable  office wear black trendy suits cool buttons grey suit trousers women girl trouser stretch different qua tapered quaclothing pair work streamlined natural western formals blazer brand office look cotton clean wood suit'

In [ ]:
new_train = train_df.drop_duplicates(subset=None, keep='first')

In [ ]:
print(new_train.shape)
print(train_df.shape)

(13108, 3)
(13132, 3)


## TFIDF

### Training

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# TF-IDF feature extractor
tfidf = TfidfVectorizer()
tfidf.fit(new_train['catalog_new_tags'])
tfidf_recipe = tfidf.transform(new_train['catalog_new_tags'])

In [ ]:
tfidf_recipe

<13108x11859 sparse matrix of type '<class 'numpy.float64'>'
	with 642489 stored elements in Compressed Sparse Row format>

In [ ]:
def get_recommendations(n_recs,rec):
    ingredients_tfidf = tfidf.transform([rec])
    cos_sim = map(lambda x: cosine_similarity(ingredients_tfidf, x), tfidf_recipe)
    scores = list(cos_sim)
    new_df = new_train[['catalog_id','catalog_name']].copy()
    new_df['scores'] = scores
    new_df['scores'] = new_df['scores'].apply(lambda x: x[0][0])
    new_df = new_df.drop_duplicates(subset=['catalog_name'])
    new_df = new_df.sort_values(by='scores',ascending=False)[:n_recs]
    return new_df

# a = '''
# qua stylish pockets workwear suit cut boss lady trendy slanted silhouette cool suits different streamlined girl clean
#  corporatefashion stretch grey suit designer work wear pair tapered fashionable  blazer office wear work girls flexible
#   woven trousers formals Stretch Wool Tailored Suit - Grey  women ways brand
#  cotton durable black buttons formal show pants wear quaclothing office look western professional
#   boss lady look fashion  wood style trouser fashionista natural officelook western wear
# '''
# get_recommendations(5,a)

### Prediction

In [ ]:
content[8]

{'_id': {'$oid': '*****************bbe0'},
 'brand_ids': [{'$oid': '*****************11a9'}],
 'caption': 'Keep your skin hydrated with @prolixr All-New Vitamin-C Serum💥 #prolixrbeauty #prolixr #skincare #hydration',
 'catalog_ids': [{'$oid': '5fbb77d8c94eac4d3ca46091'}],
 'catalog_info': None,
 'category_path': ['/615d82cd528b706ef3882b31/615eb2ee528b706ef3883376'],
 'comment_count': 0,
 'created_at': {'$date': '2021-04-21T21:41:51.915Z'},
 'hashtags': ['#prolixrbeauty', '#prolixr', '#skincare', '#hydration'],
 'influencer_ids': [{'$oid': '*****************f5c3'}],
 'is_active': True,
 'is_processed': True,
 'label': {'age_groups': ['26-30', '30-35'],
  'genders': ['girl'],
  'interests': ['skincare',
   'beauty',
   'makeup',
   'looking good',
   'routine',
   'vitamin c',
   'prolixr']},
 'last_sync': {'$date': '2022-02-15T05:38:50.576Z'},
 'like_count': 1,
 'like_ids': [{'$oid': '*****************e85c'}],
 'liked_by': [{'$oid': '*****************e01f'}],
 'media_type': 'video',
 '

In [ ]:
caption = 'Keep your skin hydrated with @prolixr All-New Vitamin-C Serum💥 #prolixrbeauty #prolixr #skincare #hydration'
interests =['skincare',
   'beauty',
   'makeup',
   'looking good',
   'routine',
   'vitamin c',
   'prolixr']
genders = ['girl']
tags = ['prolixrbeauty', 'prolixr', 'skincare', 'hydration']
caption = preprocess_sent(caption)

h = []
h.extend(interests)
h.extend(caption)
h.extend(genders)
h.extend(tags)
h = list(set(h))


In [ ]:

input_ = ' '.join(i for i in h)
get_recommendations(5,input_)

,catalog_id,catalog_name,scores
7,5fbb77d8c94eac4d3ca46091,Beauty Shield Vitamin C Serum,0.632338
433,5f5def66b35d64d69f1b93e7,Vitamin C Face Serum,0.163210
1705,60117f8a52e78c76b023fec5,Perfect Skin Bundle Kit,0.148477
572,6107a33f8a16dc68df17e427,Vitamin C Professional Face Serum,0.142155
572,6105426e8a16dc68df17e3a3,Vitamin C Brightening Face Wash,0.138250


## WORD2VEC

### Training

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(train_df['catalog_new_tags'],min_count=1,sg=1)


In [ ]:
def get_recommendations_wrdvec(n_recs,rec):
    ingredients_tfidf = tfidf.transform([rec])
    cos_sim = map(lambda x: cosine_similarity(ingredients_tfidf, x), tfidf_recipe)
    scores = list(cos_sim)
    new_df = new_train[['catalog_id','catalog_name']].copy()
    new_df['scores'] = scores
    new_df['scores'] = new_df['scores'].apply(lambda x: x[0][0])
    new_df = new_df.drop_duplicates(subset=['catalog_name'])
    new_df = new_df.sort_values(by='scores',ascending=False)[:n_recs]
    return new_df

# a = '''
# qua stylish pockets workwear suit cut boss lady trendy slanted silhouette cool suits different streamlined girl clean
#  corporatefashion stretch grey suit designer work wear pair tapered fashionable  blazer office wear work girls flexible
#   woven trousers formals Stretch Wool Tailored Suit - Grey  women ways brand
#  cotton durable black buttons formal show pants wear quaclothing office look western professional
#   boss lady look fashion  wood style trouser fashionista natural officelook western wear
# '''
# get_recommendations_wrdvec(5,a)

### Predictions

In [ ]:
caption = 'Keep your skin hydrated'
interests =['skincare',
   'beauty',
   'makeup',
   'looking good',
   'routine',
   'vitamin c',
   'prolixr']
genders = ['girl']
tags = ['prolixrbeauty', 'prolixr', 'skincare', 'hydration']
caption = preprocess_sent(caption)

h = []
h.extend(interests)
h.extend(caption)
h.extend(genders)
h.extend(tags)
h = list(set(h))


In [ ]:
input_ = ' '.join(i for i in h)
get_recommendations_wrdvec(5,input_)

,catalog_id,catalog_name,scores
7,5fbb77d8c94eac4d3ca46091,Beauty Shield Vitamin C Serum,0.632338
433,5f5def66b35d64d69f1b93e7,Vitamin C Face Serum,0.163210
1705,60117f8a52e78c76b023fec5,Perfect Skin Bundle Kit,0.148477
572,6107a33f8a16dc68df17e427,Vitamin C Professional Face Serum,0.142155
572,6105426e8a16dc68df17e3a3,Vitamin C Brightening Face Wash,0.138250


## KNN

### Training

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

In [ ]:
knn1 = KNeighborsClassifier()
knn1.fit(tfidf_recipe,new_train['catalog_name'])

KNeighborsClassifier()

### prediction

In [ ]:
caption = 'Keep your skin hydrated'
interests =['skincare',
   'beauty',
   'makeup',
   'looking good',
   'routine',
   'vitamin c',
   'prolixr']
genders = ['girl']
tags = ['prolixrbeauty', 'prolixr', 'skincare', 'hydration']
caption = preprocess_sent(caption)

h = []
h.extend(interests)
h.extend(caption)
h.extend(genders)
h.extend(tags)
h = list(set(h))


In [ ]:
new = tfidf.transform(h)

In [ ]:
knn1.predict(new)

array(['', 'Good On You Hydra Matte Lipstick - Wyld Rose',
       'Arz Kiya Hai Unisex Hoodie', '', ' Coffee & Love Scrub',
       'Beauty Shield Vitamin C Serum', 'Aqua Marine Foaming Cleanser',
       'Very Berry Moisturiser',
       'Pout Potion Liquid Matte Lipstick - Girl Boss',
       'Arz Kiya Hai Unisex Hoodie', 'Beauty Shield Vitamin C Serum', '',
       ''], dtype=object)